In [3]:
import gradio as gr
import numpy as np
import requests
import pandas as pd
import hopsworks
import joblib
import torch
from torch import nn


import os
from dotenv import load_dotenv
import httpx
import datetime
import json
from urllib.request import Request, urlopen
import random
from datetime import datetime
import missingno as msno
from sklearn.preprocessing import StandardScaler






/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
project = hopsworks.login()
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/194714
Connected. Call `.close()` to terminate connection gracefully.


In [5]:
#Load api keys
load_dotenv()
weather_api_key = os.getenv("weather_api_key")
pressure_api_key = os.getenv("pressure_api_key")
flight_api_key = os.getenv("flight_api_key")

In [6]:
#Mappings
icao_to_iata_map = {
    "KDTW": "DTW",
    "KLAS": "LAS",
    "KPHL": "PHL",
    "KDEN": "DEN",
    "KCLT": "CLT",
    "KSEA": "SEA",
    "KMCO": "MCO",
    "KFLL": "FLL",
    "KIAD": "IAD",
    "KIAH": "IAH",
    "KSFO": "SFO",
    "KEWR": "EWR",
    "KMIA": "MIA",
    "KJFK": "JFK",
    "KLAX": "LAX",
    "KORD": "ORD",
    "KATL": "ATL",
}
iata_to_icao_map = {v: k for k, v in icao_to_iata_map.items()}
wac_map = {
    "BOS": 13,
    "CLT": 36,
    "DEN": 82,
    "DTW": 43,
    "EWR": 21,
    "FLL": 33,
    "IAD": 38,
    "IAH": 74,
    "JFK": 22,
    "LAS": 85,
    "LAX": 91,
    "MCO": 33,
    "MIA": 33,
    "ORD": 41,
    "PHL": 23,
    "SEA": 93,
    "SFO": 91,
    "ATL": 34,
}
weather_features = [
    ("dewpoint", "value"),
    "relative_humidity",
    ("remarks_info", "precip_hourly", "value"),
    ("remarks_info", "temperature_decimal", "value"),
    ("visibility", "value"),
    ("wind_direction", "value"),
    ("wind_gust", "value"),
    ("wind_speed", "value"),
]
pressure_features = [("pressure", "hg")]
flight_features = [
    "flight_date",
    ("departure", "iata"),
    ("departure", "delay"),
    ("departure", "scheduled"),
    ("arrival", "iata"),
    ("arrival", "delay"),
    ("arrival", "scheduled"),
]
airport_id_map={
    "CLT": 11057,
    "DEN": 11292,
    "DTW": 11433,
    "EWR": 11618,
    "FLL": 11697,
    "IAD": 12264,
    "IAH": 12266,
    "JFK": 12478,
    "LAS": 12889,
    "LAX": 12892,
    "MCO": 13204,
    "MIA": 13303,
    "ORD": 13930,
    "PHL": 14100,
    "SEA": 14747,
    "SFO": 14771,
    "ATL": 10397,
}

# Create predefined lists for origin and destination airport codes
airports = [ "PHL - PHILADELPHIA INTERNATIONAL AIRPORT, PA US",
    "SEA - SEATTLE TACOMA AIRPORT, WA US",
    "JFK - JFK INTERNATIONAL AIRPORT, NY US",
    "DEN - DENVER INTERNATIONAL AIRPORT, CO US",
    "EWR - NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",
    "LAS - MCCARRAN INTERNATIONAL AIRPORT, NV US",
    "MCO - ORLANDO INTERNATIONAL AIRPORT, FL US",
    "ATL - ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPORT, GA US",
    "FLL - FORT LAUDERDALE INTERNATIONAL AIRPORT, FL US",
    "DTW - DETROIT METRO AIRPORT, MI US",
    "IAD - WASHINGTON DULLES INTERNATIONAL AIRPORT, VA US",
    "ORD - CHICAGO OHARE INTERNATIONAL AIRPORT, IL US",
    "LAX - LOS ANGELES INTERNATIONAL AIRPORT, CA US",
    "CLT - CHARLOTTE DOUGLAS AIRPORT, NC US",
    "MIA - MIAMI INTERNATIONAL AIRPORT, FL US",
    "IAH - HOUSTON INTERCONTINENTAL AIRPORT, TX US",
    "SFO - SAN FRANCISCO INTERNATIONAL AIRPORT, CA US"]

In [7]:

#Class definition needed due to the way pytorch neural networks are saved and loaded by python
# A solution, if needed, would be to save the state dict of the NN and load the model via load_state_dict
class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(128, 64)
        self.output = nn.Linear(64, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.output(x)
        return x

#Load model from model registry
mr = project.get_model_registry()
model = mr.get_model("flight_delay_model", version=1)
model_dir = model.download()
model = joblib.load(model_dir + "/flight_delay_model.pkl")

# get the original train test splits used for training the model and use it for fitting scaler
feature_view = fs.get_feature_view(name="flight_data_v2",version=1)
X_train, X_test, y_train, y_test = feature_view.get_train_test_split(training_dataset_version=8)

#fit scaler the same way it was used for training
scaler = StandardScaler()
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
X_train_scaled = scaler.fit_transform(X_train_tensor)
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)


Connected. Call `.close()` to terminate connection gracefully.
Finished: Reading data from Hopsworks, using ArrowFlight (18.04s) 


In [8]:
def get_weather_data(selected_airports_iata):
    # Input: list of selected airports in IATA code
    # Make API call to fetch weather data for the airport
    # Process and return weather data
    responses = {}
    for airport in selected_airports_iata:
        print(f"Getting weather for {airport}")
        request = Request(
            f"https://avwx.rest/api/metar/{iata_to_icao_map[airport]}",
            headers={"Authorization": weather_api_key},
        )
        response_body = urlopen(request).read()
        response_json = json.loads(response_body)
        responses[airport] = response_json

    weather_data = []

    for airport in selected_airports_iata:
        response_json = responses[airport]
        data = {"airport": airport}
        data["HourlyDewPointTemperature"] = response_json["remarks_info"][
            "dewpoint_decimal"
        ]["value"]
        data["HourlyRelativeHumidity"] = response_json["relative_humidity"]
        if response_json["remarks_info"]["precip_hourly"] is not None:
            data["HourlyPrecipitation"] = response_json["remarks_info"]["precip_hourly"][
                "value"
            ]
        else:
            data["HourlyPrecipitation"] = 0
        data["HourlyDryBulbTemperature"] = response_json["remarks_info"][
            "temperature_decimal"
        ]["value"]
        data["HourlyVisibility"] = response_json["visibility"]["value"]
        data["HourlyWindDirection"] = response_json["wind_direction"]["value"]
        if response_json["wind_gust"] is not None:
            data["HourlyWindGustSpeed"] = response_json["wind_gust"]["value"]
        else:
            data["HourlyWindGustSpeed"] = 0
        data["HourlyWindSpeed"] = response_json["wind_speed"]["value"]
        weather_data.append(data)

    weather_data = pd.DataFrame(weather_data)
    #weather_data.info()
    return weather_data

In [9]:
def get_pressure_data(selected_airports_iata):
    # Input: list of selected airports in IATA code
    responses={}
    url = "https://api.checkwx.com/metar/KJFK/decoded"

    #response = requests.request("GET", url, headers={"X-API-Key": pressure_api_key})
    for airport in selected_airports_iata:
        print(f"Getting pressure for {airport}")
        request = Request(
            f"https://api.checkwx.com/metar/{iata_to_icao_map[airport]}/decoded",
            headers={"X-API-Key": pressure_api_key},
        )
        response_body = urlopen(request).read()
        response_json = json.loads(response_body)
        responses[airport] = response_json

    pressure_data = []

    for airport in selected_airports_iata:
        response_json = responses[airport]
        data = {"airport": airport}
        data["HourlyStationPressure"] = response_json["data"][0]["barometer"]["hg"]
        pressure_data.append(data)

    pressure_data = pd.DataFrame(pressure_data)
    #pressure_data.info()
    return pressure_data

In [10]:
def get_flight_data(origin, destination,scheduled_dep_time, scheduled_arr_time):
    # Input: origin airport IATA code, destination airport IATA code, 
    # and dep and arr time in HH:MM 24 hour format
    current_datetime = datetime.now()

    # Extract different date-related information
    day_of_week = current_datetime.weekday()  
    day_of_month = current_datetime.day  
    year = current_datetime.year  
    month = current_datetime.month



    origin_wac = wac_map[origin]
    origin_airport_id = airport_id_map[origin]

    # Mapping destination to dest_WAC and dest_airport_id
    dest_wac = wac_map[destination]
    dest_airport_id = airport_id_map[destination]
    # Create a DataFrame for the given airport codes
    airport_df = pd.DataFrame({
        "Year":[year],
        "month":[month],
        "Day_of_month":[day_of_month],
        "Day_of_week":[day_of_week],
        "origin": [origin],
        "origin_airport_id": [origin_airport_id],
        "origin_WAC": [origin_wac],
        "dest": [destination],
        "dest_airport_id": [dest_airport_id],
        "dest_WAC": [dest_wac],
        "CRS_DEP_TIME":[int(scheduled_dep_time.replace(":", ""))],
        "CRS_ARR_TIME":[int(scheduled_arr_time.replace(":", ""))],
        "airport":[origin]
        
    })

    #print(airport_df.info())
    #print(airport_df)
    return airport_df

In [11]:
# Define the function to predict flight delay based on user inputs
def predict_delay(origin, destination,scheduled_dep_time, scheduled_arr_time):
    
    #test code to try running Gradio app
    origin=origin.split()[0]
    destination=destination.split()[0]
    
    #error handling
    try:
        # check if correct hour format by trying to convert to datetime objects
        datetime.strptime(scheduled_dep_time, "%H:%M")
        datetime.strptime(scheduled_arr_time, "%H:%M")
    except ValueError:
        # else error
        return "Error: Please enter scheduled departure and arrival times in 24-hour format (HH:MM)."
    if origin == destination:
        return "Error: Origin and destination airports cannot be the same. Please select different airports."
    
    #Get data from APIs
    selected_airports_iata = [origin,destination]
    weather_data=get_weather_data(selected_airports_iata)
    pressure_data=get_pressure_data(selected_airports_iata)
    flight_data=get_flight_data(origin, destination,scheduled_dep_time, scheduled_arr_time)

    #Merge data
    weather_delay_data = pd.merge(pressure_data, weather_data, on="airport")

    # fix order of columns so that it is same as in training
    weather_delay_data=weather_delay_data.reindex(sorted(weather_delay_data.columns), axis=1)
    
    #merge columns
    flight_weather_data=pd.merge(flight_data, weather_delay_data, on="airport")
    
    #drop objects
    flight_weather_data.drop(columns=['airport', 'origin', 'dest'], inplace=True)

    #fix type
    columns_to_float64 = ['HourlyPrecipitation', 'HourlyVisibility', 'HourlyWindGustSpeed', 'HourlyWindSpeed']
    for column in columns_to_float64:
        # Convert to int64
        flight_weather_data[column] = flight_weather_data[column].astype('float64')

    #flight_weather_data.info()

    flight_weather_data=torch.tensor(flight_weather_data.values, dtype=torch.float32)
    print(flight_weather_data)
    #flight_weather_data=scaler.transform(flight_weather_data.reshape(1, -1))
    flight_weather_data=scaler.transform(flight_weather_data)

    print(flight_weather_data)
    # transform np array to torch tensor
    flight_weather_data_tensor=torch.tensor(flight_weather_data, dtype=torch.float32)
    print(flight_weather_data_tensor)

    output=model(flight_weather_data_tensor)
    #return output
    return "Predicted delay for {} to {} with the scheduled departure time {} and scheduled " \
        "arrival time {} is {} minutes".format(origin, destination, scheduled_dep_time, scheduled_arr_time,int(output.item()))


In [12]:
#Test input
origin=airports[-3]
destination=airports[4]
scheduled_dep_time="17:00"
scheduled_arr_time="20:09"

In [13]:
predict_delay(origin, destination,scheduled_dep_time, scheduled_arr_time)

Getting weather for MIA
Getting weather for EWR
Getting pressure for MIA
Getting pressure for EWR
tensor([[2.0240e+03, 1.0000e+00, 2.0000e+00, 1.0000e+00, 1.3303e+04, 3.3000e+01,
         1.1618e+04, 2.1000e+01, 1.7000e+03, 2.0090e+03, 1.0000e+01, 2.0600e+01,
         0.0000e+00, 5.0599e-01, 3.0200e+01, 1.0000e+01, 3.0000e+02, 0.0000e+00,
         6.0000e+00]])
[[  4.95473233  -1.70103969  -1.57707557  -1.49707203   0.56936602
   -0.7632732   -0.73285178  -1.21459496   0.69166931   0.92991071
    0.10298609   0.27413197  -0.09959139  -2.76975272 -20.07063159
   -1.78493149   1.13236999  -0.35591833   0.76915281]]
tensor([[  4.9547,  -1.7010,  -1.5771,  -1.4971,   0.5694,  -0.7633,  -0.7329,
          -1.2146,   0.6917,   0.9299,   0.1030,   0.2741,  -0.0996,  -2.7698,
         -20.0706,  -1.7849,   1.1324,  -0.3559,   0.7692]])


'Predicted delay for MIA to EWR with the scheduled departure time 17:00 and scheduled arrival time 20:09 is 279 minutes'

In [ ]:
# Create Gradio interface with dropdowns for airport selection
iface = gr.Interface(
    fn=predict_delay,
    inputs=[
        gr.inputs.Dropdown(choices=airports, label="Origin Airport"),
        gr.inputs.Dropdown(choices=airports, label="Destination Airport"),
        gr.inputs.Textbox(type="text", label="Enter scheduled Departure time in 24-hour format HH:MM(eg. 17:59)"),
        gr.inputs.Textbox(type="text", label="Enter scheduled Arrival time in 24-hour format HH:MM (eg. 20:59)"),
    ],
    outputs=gr.outputs.Textbox(label="Predicted Departure Delay")
)

# Launch the Gradio app
iface.launch()

## Testing inference

In [53]:
def flight_api(selected_airports_iata):
    endpoint = "http://api.aviationstack.com/v1/flights"
    flight_data = []


    for i in range(len(selected_airports_iata)):
        print(f"Getting flights for {selected_airports_iata[i]}")
        # Parameters for the API request
        rand_int = random.randint(0, len(selected_airports_iata) - 1)
        params = {
            "access_key": flight_api_key,
            "dep_iata": selected_airports_iata[
                i
            ],  # Replace 'SFO' with your desired airport code
            "arr_iata": selected_airports_iata[rand_int],
            "flight_status": ["active"]
        }

        # Making the GET request
        response = requests.get(endpoint, params=params)
        # Checking if the request was successful (status code 200)
        if response.status_code == 200:
            response = response.json()  # Parsing the JSON response
            # Handle and process 'data' (flight information)
            data = {"airport": selected_airports_iata[i]}
            if len(response["data"]) == 0:
                continue
            
            rand_int = random.randint(0, len(response["data"]) - 1)

            response = response["data"][rand_int]

            
            data["DEPARTURE_DATETIME"] = response["departure"]["scheduled"]
            data["DEPARTURE_DATETIME"] = pd.to_datetime(
                data["DEPARTURE_DATETIME"]
            )
            current_datetime = datetime.now()

            # Extract different date-related information
            day_of_week = current_datetime.weekday()  
            day_of_month = current_datetime.day  
            year = current_datetime.year  
            month = current_datetime.month
            # add year, quarter, month, day_of_month, day_of_week
            data["YEAR"] = year
            #weather_delay_data["QUARTER"] = weather_delay_data["FL_DATE"].dt.quarter
            data["MONTH"] = month
            data["DAY_OF_MONTH"] = day_of_month
            data["DAY_OF_WEEK"] = day_of_week


            data["ORIGIN"] = response["departure"]["iata"]
            # Mapping IATA codes to airport IDs for 'dest' and 'origin' columns
            
            data['origin_airport_id'] = airport_id_map[data['ORIGIN']]
            data["ORIGIN_WAC"] = wac_map[data["ORIGIN"]]
            # add wac code to weather_delay_data
            data["DEST"] = response["arrival"]["iata"]


            data['dest_airport_id'] = airport_id_map[data['DEST']]
            data["DEST_WAC"] = wac_map[data["DEST"]]
            



            
            
            
            
            

            # Transform date columns to datetime
            data["CRS_DEP_TIME"] = response["departure"]["scheduled"]
            data["CRS_ARR_TIME"] = response["arrival"]["scheduled"]
            
            data["airport"]=data["ORIGIN"]
            data["origin"]=data["ORIGIN"]
            data["dest"]=data["DEST"]
            
            if response["departure"]["delay"] is not None:
                data["DEP_DELAY_NEW"] = response["departure"]["delay"]
            else:
                data["DEP_DELAY_NEW"] = 0.0
            

            flight_data.append(data)
        else:
            print(response.json())
            print("Request failed:", response.status_code)

    flight_data = pd.DataFrame(flight_data)
    flight_data.drop(columns=[ 'ORIGIN', 'DEST','DEPARTURE_DATETIME'], inplace=True)
    flight_data.info()
    return flight_data

In [43]:
selected_airports_iata = [
    "DTW",
    "LAS",
    "PHL",
    "DEN",
    "CLT",
    "SEA",
    "MCO",
    "FLL",
    "IAD",
    "IAH",
    "SFO",
    "EWR",
    "MIA",
    "JFK",
    "LAX",
    "ORD",
    "ATL",
]

In [54]:
flight_data_api=flight_api(selected_airports_iata)
weather_data=get_weather_data(selected_airports_iata)
pressure_data=get_pressure_data(selected_airports_iata)

Getting flights for DTW
Getting flights for LAS
Getting flights for PHL
Getting flights for DEN
Getting flights for CLT
Getting flights for SEA
Getting flights for MCO
Getting flights for FLL
Getting flights for IAD
Getting flights for IAH
Getting flights for SFO
Getting flights for EWR
Getting flights for MIA
Getting flights for JFK
Getting flights for LAX
Getting flights for ORD
Getting flights for ATL
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   airport             9 non-null      object             
 1   DEPARTURE_DATETIME  9 non-null      datetime64[ns, UTC]
 2   YEAR                9 non-null      int64              
 3   MONTH               9 non-null      int64              
 4   DAY_OF_MONTH        9 non-null      int64              
 5   DAY_OF_WEEK         9 non-null      int64              
 6 

In [57]:
flight_data_api.head()

,airport,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,origin_airport_id,ORIGIN_WAC,dest_airport_id,DEST_WAC,CRS_DEP_TIME,CRS_ARR_TIME,origin,dest,DEP_DELAY_NEW
0,DTW,2024,1,2,1,11433,43,11292,82,2024-01-02T08:35:00+00:00,2024-01-02T10:05:00+00:00,DTW,DEN,67
1,LAS,2024,1,2,1,12889,85,11057,36,2024-01-02T06:00:00+00:00,2024-01-02T13:15:00+00:00,LAS,CLT,6
2,MCO,2024,1,2,1,13204,33,11433,43,2024-01-02T11:01:00+00:00,2024-01-02T13:51:00+00:00,MCO,DTW,10
3,FLL,2024,1,2,1,11697,33,12892,91,2024-01-02T07:45:00+00:00,2024-01-02T10:45:00+00:00,FLL,LAX,5
4,IAD,2024,1,2,1,12264,38,12889,85,2024-01-02T08:45:00+00:00,2024-01-02T11:01:00+00:00,IAD,LAS,12


In [98]:
#Merge data
weather_delay_data = pd.merge(pressure_data, weather_data, on="airport")

# fix order of columns so that it is same as in training
weather_delay_data=weather_delay_data.reindex(sorted(weather_delay_data.columns), axis=1)

#merge columns
flight_weather_data=pd.merge(flight_data_api, weather_delay_data, on="airport")



In [99]:
#drop objects
flight_weather_data.drop(columns=['airport', 'origin', 'dest'], inplace=True)

In [100]:
flight_weather_data.info()
flight_weather_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   YEAR                       9 non-null      int64  
 1   MONTH                      9 non-null      int64  
 2   DAY_OF_MONTH               9 non-null      int64  
 3   DAY_OF_WEEK                9 non-null      int64  
 4   origin_airport_id          9 non-null      int64  
 5   ORIGIN_WAC                 9 non-null      int64  
 6   dest_airport_id            9 non-null      int64  
 7   DEST_WAC                   9 non-null      int64  
 8   CRS_DEP_TIME               9 non-null      object 
 9   CRS_ARR_TIME               9 non-null      object 
 10  DEP_DELAY_NEW              9 non-null      int64  
 11  HourlyDewPointTemperature  9 non-null      float64
 12  HourlyDryBulbTemperature   9 non-null      float64
 13  HourlyPrecipitation        9 non-null      int64  
 14

,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,origin_airport_id,ORIGIN_WAC,dest_airport_id,DEST_WAC,CRS_DEP_TIME,CRS_ARR_TIME,DEP_DELAY_NEW,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyRelativeHumidity,HourlyStationPressure,HourlyVisibility,HourlyWindDirection,HourlyWindGustSpeed,HourlyWindSpeed
0,2024,1,2,1,11433,43,11292,82,2024-01-02T08:35:00+00:00,2024-01-02T10:05:00+00:00,67,-4.4,-0.6,0,0.754634,30.14,10,240.0,0,14
1,2024,1,2,1,12889,85,11057,36,2024-01-02T06:00:00+00:00,2024-01-02T13:15:00+00:00,6,-2.8,10.0,0,0.405517,30.24,10,290.0,0,4
2,2024,1,2,1,13204,33,11433,43,2024-01-02T11:01:00+00:00,2024-01-02T13:51:00+00:00,10,3.9,15.0,0,0.473885,30.22,10,320.0,0,7
3,2024,1,2,1,11697,33,12892,91,2024-01-02T07:45:00+00:00,2024-01-02T10:45:00+00:00,5,10.0,20.6,0,0.505993,30.17,10,350.0,0,7
4,2024,1,2,1,12264,38,12889,85,2024-01-02T08:45:00+00:00,2024-01-02T11:01:00+00:00,12,-3.9,5.6,0,0.504163,30.17,10,310.0,0,14


In [101]:
# Convert columns to datetime
flight_weather_data['CRS_DEP_TIME'] = pd.to_datetime(flight_weather_data['CRS_DEP_TIME'])
flight_weather_data['CRS_ARR_TIME'] = pd.to_datetime(flight_weather_data['CRS_ARR_TIME'])

# Extract time in HHMM format without a colon
flight_weather_data['CRS_DEP_TIME'] = flight_weather_data['CRS_DEP_TIME'].dt.strftime('%H%M')
flight_weather_data['CRS_ARR_TIME'] = flight_weather_data['CRS_ARR_TIME'].dt.strftime('%H%M')

In [102]:
columns_to_int64 = ['CRS_ARR_TIME', 'CRS_DEP_TIME', 'YEAR', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK']
columns_to_float64 = ['HourlyPrecipitation', 'HourlyVisibility', 'HourlyWindGustSpeed', 'HourlyWindSpeed', 'DEP_DELAY_NEW']

for column in columns_to_int64:
    # Convert to int64
    flight_weather_data[column] = flight_weather_data[column].astype('int64')
for column in columns_to_float64:
    # Convert to int64
    flight_weather_data[column] = flight_weather_data[column].astype('float64')

In [103]:
flight_weather_data.info()
flight_weather_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   YEAR                       9 non-null      int64  
 1   MONTH                      9 non-null      int64  
 2   DAY_OF_MONTH               9 non-null      int64  
 3   DAY_OF_WEEK                9 non-null      int64  
 4   origin_airport_id          9 non-null      int64  
 5   ORIGIN_WAC                 9 non-null      int64  
 6   dest_airport_id            9 non-null      int64  
 7   DEST_WAC                   9 non-null      int64  
 8   CRS_DEP_TIME               9 non-null      int64  
 9   CRS_ARR_TIME               9 non-null      int64  
 10  DEP_DELAY_NEW              9 non-null      float64
 11  HourlyDewPointTemperature  9 non-null      float64
 12  HourlyDryBulbTemperature   9 non-null      float64
 13  HourlyPrecipitation        9 non-null      float64
 14

,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,origin_airport_id,ORIGIN_WAC,dest_airport_id,DEST_WAC,CRS_DEP_TIME,CRS_ARR_TIME,DEP_DELAY_NEW,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyRelativeHumidity,HourlyStationPressure,HourlyVisibility,HourlyWindDirection,HourlyWindGustSpeed,HourlyWindSpeed
0,2024,1,2,1,11433,43,11292,82,835,1005,67.0,-4.4,-0.6,0.0,0.754634,30.14,10.0,240.0,0.0,14.0
1,2024,1,2,1,12889,85,11057,36,600,1315,6.0,-2.8,10.0,0.0,0.405517,30.24,10.0,290.0,0.0,4.0
2,2024,1,2,1,13204,33,11433,43,1101,1351,10.0,3.9,15.0,0.0,0.473885,30.22,10.0,320.0,0.0,7.0
3,2024,1,2,1,11697,33,12892,91,745,1045,5.0,10.0,20.6,0.0,0.505993,30.17,10.0,350.0,0.0,7.0
4,2024,1,2,1,12264,38,12889,85,845,1101,12.0,-3.9,5.6,0.0,0.504163,30.17,10.0,310.0,0.0,14.0


In [104]:
target_variable = flight_weather_data.pop('DEP_DELAY_NEW')

In [105]:
flight_weather_data.info()
flight_weather_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   YEAR                       9 non-null      int64  
 1   MONTH                      9 non-null      int64  
 2   DAY_OF_MONTH               9 non-null      int64  
 3   DAY_OF_WEEK                9 non-null      int64  
 4   origin_airport_id          9 non-null      int64  
 5   ORIGIN_WAC                 9 non-null      int64  
 6   dest_airport_id            9 non-null      int64  
 7   DEST_WAC                   9 non-null      int64  
 8   CRS_DEP_TIME               9 non-null      int64  
 9   CRS_ARR_TIME               9 non-null      int64  
 10  HourlyDewPointTemperature  9 non-null      float64
 11  HourlyDryBulbTemperature   9 non-null      float64
 12  HourlyPrecipitation        9 non-null      float64
 13  HourlyRelativeHumidity     9 non-null      float64
 14

,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,origin_airport_id,ORIGIN_WAC,dest_airport_id,DEST_WAC,CRS_DEP_TIME,CRS_ARR_TIME,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyRelativeHumidity,HourlyStationPressure,HourlyVisibility,HourlyWindDirection,HourlyWindGustSpeed,HourlyWindSpeed
0,2024,1,2,1,11433,43,11292,82,835,1005,-4.4,-0.6,0.0,0.754634,30.14,10.0,240.0,0.0,14.0
1,2024,1,2,1,12889,85,11057,36,600,1315,-2.8,10.0,0.0,0.405517,30.24,10.0,290.0,0.0,4.0
2,2024,1,2,1,13204,33,11433,43,1101,1351,3.9,15.0,0.0,0.473885,30.22,10.0,320.0,0.0,7.0
3,2024,1,2,1,11697,33,12892,91,745,1045,10.0,20.6,0.0,0.505993,30.17,10.0,350.0,0.0,7.0
4,2024,1,2,1,12264,38,12889,85,845,1101,-3.9,5.6,0.0,0.504163,30.17,10.0,310.0,0.0,14.0


In [106]:

target_variable.info()
target_variable.head()

<class 'pandas.core.series.Series'>
RangeIndex: 9 entries, 0 to 8
Series name: DEP_DELAY_NEW
Non-Null Count  Dtype  
--------------  -----  
9 non-null      float64
dtypes: float64(1)
memory usage: 200.0 bytes


0    67.0
1     6.0
2    10.0
3     5.0
4    12.0
Name: DEP_DELAY_NEW, dtype: float64

In [120]:
model.eval()

# Perform inference
with torch.no_grad():
    print(len(target_variable))
    for i in range(len(target_variable)):
        inference_data = torch.tensor(flight_weather_data.iloc[i].values, dtype=torch.float32)
        inference_data = scaler.transform(inference_data.reshape(1, -1))
        inference_data = torch.tensor(inference_data, dtype=torch.float32)
        #print(inference_data)
        y_pred=model(inference_data)
        print("Predicted departure delay in minutes: " +str(y_pred.item())+". Real departure delay from flight API: "+str(target_variable.iloc[i]))

9
Predicted departure delay in minutes: 169.76779174804688. Real departure delay from flight API: 67.0
Predicted departure delay in minutes: 161.4446258544922. Real departure delay from flight API: 6.0
Predicted departure delay in minutes: 232.5779266357422. Real departure delay from flight API: 10.0
Predicted departure delay in minutes: 208.79269409179688. Real departure delay from flight API: 5.0
Predicted departure delay in minutes: 187.9297637939453. Real departure delay from flight API: 12.0
Predicted departure delay in minutes: 186.5002899169922. Real departure delay from flight API: 25.0
Predicted departure delay in minutes: nan. Real departure delay from flight API: 29.0
Predicted departure delay in minutes: 160.97447204589844. Real departure delay from flight API: 14.0
Predicted departure delay in minutes: 217.53977966308594. Real departure delay from flight API: 28.0


In [121]:
flight_weather_data.loc[:, 'YEAR'] = 2022

In [122]:
model.eval()

# Perform inference
with torch.no_grad():
    print(len(target_variable))
    for i in range(len(target_variable)):
        inference_data = torch.tensor(flight_weather_data.iloc[i].values, dtype=torch.float32)
        inference_data = scaler.transform(inference_data.reshape(1, -1))
        inference_data = torch.tensor(inference_data, dtype=torch.float32)
        #print(inference_data)
        y_pred=model(inference_data)
        print("Predicted departure delay in minutes: " +str(y_pred.item())+". Real departure delay from flight API: "+str(target_variable.iloc[i]))

9
Predicted departure delay in minutes: 109.66477966308594. Real departure delay from flight API: 67.0
Predicted departure delay in minutes: 106.05193328857422. Real departure delay from flight API: 6.0
Predicted departure delay in minutes: 165.35415649414062. Real departure delay from flight API: 10.0
Predicted departure delay in minutes: 146.67547607421875. Real departure delay from flight API: 5.0
Predicted departure delay in minutes: 125.67322540283203. Real departure delay from flight API: 12.0
Predicted departure delay in minutes: 124.69072723388672. Real departure delay from flight API: 25.0
Predicted departure delay in minutes: nan. Real departure delay from flight API: 29.0
Predicted departure delay in minutes: 117.04496765136719. Real departure delay from flight API: 14.0
Predicted departure delay in minutes: 155.86827087402344. Real departure delay from flight API: 28.0
